In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib as plt
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

from tensorflow import keras
#import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from keras import backend as K
import gensim

/home/medic/python36/lib64/python3.6/site-packages/OpenSSL/crypto.py:8: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography import utils, x509
[nltk_data] Downloading package punkt to /home/medic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Подготовка данных для обучения моделей

In [3]:
test_data = pd.read_excel('./test_hotels.xlsx')
train_data = pd.read_excel('./train_hotels.xlsx')

In [4]:
train_data.head()

,sentiment,text
0,4,"Очень достойный отель с прекрасными номерами, ..."
1,4,"Остановились в Барселоне проездом, т.к. нужно ..."
2,4,Типичная сетевая гостиница. Главный плюс-шикар...
3,1,"Начнем с того, что в этом отеле не берут деньг..."
4,5,"Отель находится в отдалении от центра,но пешко..."


In [5]:
def create_sentiment(x):
    sentiment = 0
    if x > 3:
        sentiment = 1
    else:
        sentiment = 0
    
    return sentiment

# create class from digital order
train_data['class'] = train_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
test_data['class'] = test_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
# clear Nan data in datasets
train_data.dropna(subset = ['text'], inplace = True)
test_data.dropna(subset = ['text'], inplace = True)

In [6]:
train_data.head(15)

,sentiment,text,class
0,4,"Очень достойный отель с прекрасными номерами, ...",1
1,4,"Остановились в Барселоне проездом, т.к. нужно ...",1
2,4,Типичная сетевая гостиница. Главный плюс-шикар...,1
3,1,"Начнем с того, что в этом отеле не берут деньг...",0
4,5,"Отель находится в отдалении от центра,но пешко...",1
5,5,Приехали с сестрой и её мужем на машине и с со...,1
6,5,Чистота и удобство в номерах – по высшему клас...,1
7,5,В сети отелей NH Collection отдыхаем впервые. ...,1
8,5,"У отеля неплохое расположение, рядом есть разл...",1
9,3,отель соответствует заявленным звездам.номера ...,0


In [7]:
# preproccesing a data 
stopwords = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def clean_text(text):
    text = str(text)
    text = text.lower()
    text = [morpher.parse(word)[0].normal_form for word in text.split() if word not in stopwords]
    return " ".join(text)

train_data['text'] = train_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].apply(clean_text)

In [8]:
train_data.head(6)

,sentiment,text,class
0,4,"достойный отель прекрасный номерами, хороший и...",1
1,4,"остановиться барселона проездом, т.к. посетить...",1
2,4,типичный сетевой гостиница. главный плюс-шикар...,1
3,1,"начать того, отель брать деньга воздух. звонок...",0
4,5,"отель находиться отдаление центра,ный пешком д...",1
5,5,приехать сестра муж машина собакой. парковка п...,1


In [9]:
train_data['class'].value_counts()

1    41507
0     8668
Name: class, dtype: int64

У нас дисбаланс классов. Это не есть хорошо.

In [10]:
train_corpus = " ".join(train_data['text'])
train_corpus = train_corpus.lower()
test_corpus = " ".join(test_data['text'])
test_corpus = test_corpus.lower()

tokens = word_tokenize(train_corpus)

max_words = 300
max_len = 40

tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:15]

['отель',
 'номер',
 'завтрак',
 'хороший',
 'персонал',
 'минута',
 'вид',
 'метро',
 'отличный',
 'расположение',
 'большой',
 'центр',
 'находиться',
 'день',
 'ресторан']

In [11]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [12]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [13]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_data["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_data["text"]], dtype=np.int32)

In [14]:
x_train.shape, x_test.shape

((50175, 40), (6876, 40))

In [15]:
x_train[13]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   4,   1,   2,  35,  10,  30, 295,
         6,   6,   8,  74,  51,   6, 123, 118,  45, 208, 107,   1, 113,
         8], dtype=int32)

In [16]:
num_classes = 2
y_train = keras.utils.to_categorical(train_data['class'], num_classes)
y_test = keras.utils.to_categorical(test_data['class'], num_classes)

#### Пробуем CNN архитектуру

In [31]:
epochs = 80
batch_size = 512
print_batch_n = 100

cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
cnn_model.add(Conv1D(128, 3))
cnn_model.add(Activation("relu"))
cnn_model.add(GlobalMaxPool1D())
cnn_model.add(Dense(10))
cnn_model.add(Activation("relu"))
cnn_model.add(Dense(num_classes))
cnn_model.add(Activation('softmax'))

In [18]:
# for calculate f1 metric it that we have disbalance classes
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [32]:
cnn_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = [f1])

In [33]:
%%time
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_f1')  

history_cnn = cnn_model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/80
89/89 [==============================] - 8s 85ms/step - loss: 0.4130 - f1: 0.8375 - val_loss: 0.3855 - val_f1: 0.8355
CPU times: user 19.7 s, sys: 376 ms, total: 20.1 s
Wall time: 8.16 s


In [34]:
score = cnn_model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test f1:', score[1])

14/14 [==============================] - 0s 21ms/step - loss: 0.3302 - f1: 0.8622


Test f1: 0.8621955513954163


#### Теперь построим сетку RNN

In [38]:
model_rnn = Sequential()

model_rnn.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_rnn.add(Masking(mask_value=0.0))
model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(64, activation='relu'))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[f1])

In [57]:
%%time
early_stopping_rnn=EarlyStopping(monitor='val_f1')  

history = model_rnn.fit(x_train, train_data['class'].values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping_rnn])

Epoch 1/80
89/89 [==============================] - 9s 97ms/step - loss: 0.2966 - f1: 0.9282 - val_loss: 0.3694 - val_f1: 0.9066
CPU times: user 17.3 s, sys: 948 ms, total: 18.3 s
Wall time: 40.7 s


In [49]:
score_rnn = model_rnn.evaluate(x_test, test_data['class'].values, batch_size=batch_size, verbose=1)
print('\n')
print('Test f1:', score_rnn[1])

14/14 [==============================] - 0s 33ms/step - loss: 0.3157 - f1: 0.9230


Test f1: 0.9230213761329651


#### Результаты сравнения "в лоб" CNN архитектуры и RNN для классификации текстов
RNN сетка показала себя лучше, ее метрика 0.913 в противовес CNN - 0.862 и это ожидаемо, так как RNN может анализировать последовательности, что лучше для классификации текстов, так как текст это в основном последовательность. Чем длиннее последовательность тем в теории должен быть лучше результат классификации текста, так как захватывается больше контекста.

Однако не стоить думать о том, что CNN используется только для анализа картинок с котиками. CNN довольно хорошо понимает и анализирует короткие тексты. Если ширина нашего ngram 2 или 3 этого будет достаточно для анализа семантики теста, так как в отзывах часто важны несколько слов, и нам не нужно понимать длинные предложения. Достаточно распознать пару "плохой сервис" или "ужасное качество" и этого будет достаточно чтобы понять, что мы имеем место с отрицательным отзывом.
Именно по этому мы получили не плохой результат от CNN модели без всяких дополнительных парамеров.

In [51]:
model_cnn_rnn = Sequential()

model_cnn_rnn.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_cnn_rnn.add(Conv1D(128, 3))
model_cnn_rnn.add(SimpleRNN(64))
model_cnn_rnn.add(Dense(64, activation='relu'))
model_cnn_rnn.add(Dropout(0.5))
model_cnn_rnn.add(Dense(1, activation='sigmoid'))

model_cnn_rnn.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[f1])

In [58]:
%%time
early_stopping_cnn_rnn = EarlyStopping(monitor='val_f1')  

history_cnn_rnn = model_cnn_rnn.fit(x_train, train_data['class'].values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping_cnn_rnn])

Epoch 1/80
89/89 [==============================] - 11s 122ms/step - loss: 0.3126 - f1: 0.9258 - val_loss: 0.3636 - val_f1: 0.9091
CPU times: user 25 s, sys: 875 ms, total: 25.9 s
Wall time: 12.6 s


In [54]:
score_cnn_rnn = model_cnn_rnn.evaluate(x_test, test_data['class'].values, batch_size=batch_size, verbose=1)
print('\n')
print('Test f1:', score_cnn_rnn[1])

14/14 [==============================] - 0s 35ms/step - loss: 0.3285 - f1: 0.9172


Test f1: 0.9171813130378723


Комбинация CNN и RNN слоев в модели показала несколько худшие результаты чем "чистая" RNN. Скорее всего это благодаря тому, что первый CNN слой "отфильтровал" "нужные" слова и общий контект который может уловить "чистая" RNN потерялся, по этому мы имеем что-то среднее между CNN и RNN сетками. 
Что мы получили по итогу смиксовав две архитектуры:

<table>
<thead>
    <tr><th>arch</th><th>F1 (score)</th><th>Time execution</th></tr>
</thead>
<tbody>
    <tr><td>CNN</td><td>0.8621</td><td>total: 8.16 s</td></tr>
    <tr><td>RNN</td><td>0.9230</td><td>total: 40.7 s</td></tr>
    <tr><td>CNN_RNN</td><td>0.9171</td><td>total: 12.6 s</td></tr>
</tbody>
</table>